# Segurança

O Istio oferece uma solução de segurança abrangente para resolver problemas, como:

* Defender contra ataques _man-in-the-middle_;
* Fornecer controle de acesso de serviço flexível;
* Determinar quem fez o quê e quando.

![Istio security architecture](media/istio-sec-arch.png)
Istio [High-level architecture](https://istio.io/latest/docs/concepts/security/#high-level-architecture)

Neste cursos iremos abordar dois aspectos da segurança do Istio: TLS mútuo e autenticação de requisições com JWT.

## TLS mútuo

O Istio faz o túnel da comunicação serviço a serviço por meio dos _proxies_ do lado do cliente e do servidor, que são implementados com Envoy. Quando uma carga de trabalho envia uma solicitação para outra carga de trabalho usando autenticação [TLS mútua](https://istio.io/latest/docs/concepts/security/#mutual-tls-authentication), a solicitação é tratada da seguinte maneira:

1. O Istio redireciona o tráfego de saída de um cliente para o sidecar Envoy local do cliente.

2. O Envoy do lado do cliente inicia um _handshake_ TLS mútuo com o Envoy do lado do servidor. Durante o _handshake_, o Envoy do lado do cliente também faz uma verificação de nomenclatura segura para verificar se a conta de serviço apresentada no certificado do servidor está autorizada a executar o serviço de destino.

3. O Envoy do lado do cliente e o Envoy do lado do servidor estabelecem uma conexão TLS mútua, e o Istio encaminha o tráfego do Envoy do lado do cliente para o Envoy do lado do servidor.

4. Após a autorização, o Envoy do lado do servidor encaminha o tráfego para o serviço do servidor por meio de conexões TCP locais.

O Istio configura automaticamente os _sidecars_ da carga de trabalho para usar TLS mútuo. Por padrão, o Istio configura as cargas de trabalho de destino usando o modo `PERMISSIVE`. Quando o modo PERMISSIVO está habilitado, um serviço pode aceitar tanto texto simples quanto tráfego TLS. Para permitir apenas o tráfego TLS, a configuração precisa ser alterada para o modo `STRICT`.

Vamos verificar isso com ajuda do kiali

<http://localhost:20001/kiali>

Vá para o gráfico (Versioned App graph) e na caixa de seleção _Display_, marque _Security_.

Vocë deverá ver que todas as conexões agora tem um símbolo de cadeado, isso indica que as requisições são seguras (TLS).

![Kiali MTLS](media/kiali-MTLS.png)

### Desabilitando o TLS múto

Você pode utilizar a mesma confiugração para desabilitar o TLS múto, porém isso não é recomendado, a não ser que você implemente sua própria solução.

In [ ]:
# Desabilitando o MTLS
cat <<EOF | kubectl apply -f -
apiVersion: "security.istio.io/v1beta1"
kind: "PeerAuthentication"
metadata:
  name: "disable-mtls-namespace"
  namespace: "default"
spec:
  mtls:
    mode: DISABLE
EOF

Verifique novamente no Kiali, após alguns segundos, os cadeados desapareceram.

Vocë pode desligar o MTLS para toda a malha

```bash
kubectl apply -n istio-system -f - <<EOF
apiVersion: "security.istio.io/v1beta1"
kind: "PeerAuthentication"
metadata:
  name: "disable-mtls-mesh"
spec:
  mtls:
    mode: DISABLE
EOF
```
> Para excluir a configuração acima `kubectl delete pa/disable-mtls-mesh -n istio-system`

Para voltar para a configuração original, vamos excluir o PeerAuthentication. Veremos mais sobre como controlar o MTLS nas proximas seções.

In [ ]:
kubectl delete PeerAuthentication/disable-mtls-namespace

#### Desabilitando TLS múto para uma aplicação

Como a maioria das configurações, você pode especificar um seletor para o qual a configuração é válida. Essa configuraçâo pode ser necessária se vocë estiver lidando com aplicações legadas, que não utilizam o protocolo https, vocë poderá desligá-lo temporariamente, até que ela seja refatorada.

```yaml
apiVersion: "security.istio.io/v1beta1"
kind: "PeerAuthentication"
metadata:
  name: "front-end-peer-policy"
  namespace: "default"
spec:
  selector:
    matchLabels:
      app: front-end
  mtls:
    mode: DISABLE
```

Há casos em que você deseja desativar o TLS mútuo para uma porta:

```yaml
apiVersion: "security.istio.io/v1beta1"
kind: "PeerAuthentication"
metadata:
  name: "front-end-workload-policy"
  namespace: "default"
spec:
  selector:
     matchLabels:
       app: front-end
  portLevelMtls:
    8000:
      mode: DISABLE
```

Verifique se sobrou alguma configura configuração e exclua.

In [ ]:
kubectl get pa --all-namespaces

## Políticas de autenticação de solicitação

As [políticas de autenticação de solicitação](https://istio.io/latest/docs/concepts/security/#request-authentication) especificam os valores necessários para validar um [JSON Web Token](https://jwt.io/) (JWT). Esses valores incluem, entre outros, o seguinte:

* A localização do token na solicitação
* O emissor ou o pedido
* O JSON Web Key Set público (JWKS)

#### Autorização com JWT

Para configurar uma política de autorização do Istio, iremos criar uma configuração do tiop `RequestAuthentication`. Uma política de autorização do Istio oferece suporte a declarações JWT digitadas por string e por lista de strings.


In [ ]:
# request authentication policy

kubectl apply -f - <<EOF
apiVersion: "security.istio.io/v1beta1"
kind: "RequestAuthentication"
metadata:
  name: "jwt-example"
  namespace: foo
spec:
  selector:
    matchLabels:
      app: httpbin
  jwtRules:
  - issuer: "testing@secure.istio.io"
    jwksUri: "https://raw.githubusercontent.com/istio/istio/release-1.8/security/tools/jwt/samples/jwks.json"
EOF

In [ ]:
# request with an invalid JWT is denied
kubectl exec "$(kubectl get pod -l app=sleep -n foo -o jsonpath={.items..metadata.name})" -c sleep -n foo -- curl "http://httpbin.foo:8000/headers" -s -o /dev/null -w "%{http_code}\n"


In [ ]:
#  request without a JWT is allowed because there is no authorization policy
kubectl exec "$(kubectl get pod -l app=sleep -n foo -o jsonpath={.items..metadata.name})" -c sleep -n foo -- curl "http://httpbin.foo:8000/headers" -s -o /dev/null -w "%{http_code}\n"


In [ ]:
# authorization policy for the httpbin
kubectl apply -f - <<EOF
apiVersion: security.istio.io/v1beta1
kind: AuthorizationPolicy
metadata:
  name: require-jwt
  namespace: foo
spec:
  selector:
    matchLabels:
      app: httpbin
  action: ALLOW
  rules:
  - from:
    - source:
       requestPrincipals: ["testing@secure.istio.io/testing@secure.istio.io"]
EOF

In [ ]:
# JWT that sets the iss and sub keys to the same value
TOKEN=$(curl https://raw.githubusercontent.com/istio/istio/release-1.8/security/tools/jwt/samples/demo.jwt -s) && echo "$TOKEN" | cut -d '.' -f2 - | base64 --decode -


In [ ]:
# request with a valid JWT
kubectl exec "$(kubectl get pod -l app=sleep -n foo -o jsonpath={.items..metadata.name})" -c sleep -n foo -- curl "http://httpbin.foo:8000/headers" -s -o /dev/null -H "Authorization: Bearer $TOKEN" -w "%{http_code}\n"


In [ ]:
# request without a JWT
kubectl exec "$(kubectl get pod -l app=sleep -n foo -o jsonpath={.items..metadata.name})" -c sleep -n foo -- curl "http://httpbin.foo:8000/headers" -s -o /dev/null -w "%{http_code}\n"


## Bloquear tráfego de saída

Na seção 10.1.5 - Gateways, implementamos um gateway de saída, nesta seção discutiremos um dos motivos para implementá-lo, segurança.

Istio facilita a comunicação intra serviços, as vezes chamada de comunicação [leste-oeste](https://en.wikipedia.org/wiki/East-west_traffic), mas no ponto de vista de segurança, o risco está na comunicação norte-sul, ou seja, tráfico para fora da malha, por exemplo, chamada de APIs na internet.

Por padrão, o Istio não impede a comunicação dos serviços, dentro da malha, chamarem serviços externos, mesmo que não estejam registrados (_ServiceEntry_).

Vamos verificar em que modo nossa malha está configurada:

In [ ]:
kubectl get istiooperator installed-state -n istio-system -o jsonpath='{.spec.meshConfig.outboundTrafficPolicy.mode}'

Vamos bloquear todo o tráfego de saída que não está registrado:

In [ ]:
istioctl install --set profile=demo --skip-confirmation --set meshConfig.outboundTrafficPolicy.mode=REGISTRY_ONLY

Vamos verificar no kiali o que está acontecendo. <http://localhost:20001>

![Imagem ou Video do tráfego bloqueado](media/video-placeholder.jpg)

Como você pode verificar, o tráfego entre os serviços está OK, mas o tráfego para fora da malha foi bloqueado.

Vamos restaurar a configuração ao modo padrão.

In [ ]:
istioctl install --set profile=demo --skip-confirmation --set meshConfig.outboundTrafficPolicy.mode=ALLOW_ANY

## TODO

* Egress Blocking
* Access Control
* RBAC